In [1]:
# === INSTALACJA I IMPORTY ===
!pip install cryptography -q

import pandas as pd
import io
import hashlib
import uuid
import time
import logging
from cryptography.fernet import Fernet
from google.colab import files

# Konfiguracja logowania (Audyt)
logging.basicConfig(filename='access_log.txt', level=logging.INFO,
                    format='%(asctime)s - %(message)s', datefmt='%Y-%m-%d %H:%M:%S')

def log_access(user_role, action, status):
    """Funkcja logująca operacje do pliku (Symulacja audytu)"""
    log_entry = f"Rola: {user_role} | Akcja: {action} | Status: {status}"
    logging.info(log_entry)
    print(f"[AUDYT] {log_entry}")

# === ETAP 1: PRZYGOTOWANIE DANYCH (WCZYTYWANIE) ===
print("=== ETAP 1: WCZYTYWANIE DANYCH ===")
print("Proszę wybrać plik CSV z lokalnego dysku:")

uploaded = files.upload()

if not uploaded:
    print("Nie wybrano pliku. Kończę działanie.")
else:
    # Pobranie nazwy pierwszego pliku
    filename = next(iter(uploaded))

    # Wczytanie danych
    try:
        df = pd.read_csv(io.BytesIO(uploaded[filename]))
        print(f"\nSukces! Wczytano plik: {filename}")
        print(f"Rozmiar danych: {df.shape}")
        print("\nPodgląd danych surowych:")
        display(df.head())
    except Exception as e:
        print(f"Błąd wczytywania pliku: {e}")
        df = pd.DataFrame() # Pusty DF, aby kod nie wywalił błędu dalej

    if not df.empty:
        # === ETAP 2: SZYFROWANIE I INTEGRALNOŚĆ ===
        print("\n=== ETAP 2: SZYFROWANIE I INTEGRALNOŚĆ ===")

        # 2a. Generowanie klucza szyfrującego
        key = Fernet.generate_key()
        cipher_suite = Fernet(key)
        print(f"Wygenerowano klucz symetryczny: {key.decode()[:10]}...")

        # 2b. Szyfrowanie danych wrażliwych (np. Diagnoza lub Nazwisko - szukamy kolumn)
        # Próba znalezienia kolumny tekstowej do zaszyfrowania
        target_col = None
        possible_cols = ['diagnosis', 'Diagnosis', 'Condition', 'conditions', 'Nazwisko', 'Surname']

        for col in df.columns:
            if col in possible_cols:
                target_col = col
                break

        # Jeśli nie znaleziono typowej, bierzemy ostatnią kolumnę tekstową (często diagnoza)
        if not target_col:
            target_col = df.select_dtypes(include=['object']).columns[-1]

        print(f"Szyfrowanie kolumny: '{target_col}'")

        # Funkcja pomocnicza do szyfrowania
        def encrypt_text(text):
            if isinstance(text, str):
                return cipher_suite.encrypt(text.encode()).decode()
            return text

        # Kopia do pracy
        df_secure = df.copy()
        df_secure[f'{target_col}_encrypted'] = df_secure[target_col].apply(encrypt_text)

        display(df_secure[[target_col, f'{target_col}_encrypted']].head())

        # 2c. Funkcja skrótu (Hashing) i weryfikacja integralności
        def calculate_file_hash(dataframe):
            # Zamiana DF na string/bytes, aby policzyć hash całego zbioru
            data_str = dataframe.to_csv(index=False)
            return hashlib.sha256(data_str.encode()).hexdigest()

        original_hash = calculate_file_hash(df)
        print(f"\nHash oryginalnego zbioru (SHA-256): {original_hash}")

        # Symulacja ataku / uszkodzenia danych (zmiana jednego znaku)
        df_tampered = df.copy()
        # Zmieniamy jedną wartość w pierwszej komórce
        val = df_tampered.iloc[0, 0]
        if isinstance(val, (int, float)):
             df_tampered.iloc[0, 0] = val + 1
        else:
             df_tampered.iloc[0, 0] = str(val) + "X"

        tampered_hash = calculate_file_hash(df_tampered)
        print(f"Hash po modyfikacji 1 wartości:    {tampered_hash}")
        print(f"Czy hashe są równe? {original_hash == tampered_hash} (Efekt lawiny widoczny!)")

        # === ETAP 3: PSEUDONIMIZACJA I ANONIMIZACJA ===
        print("\n=== ETAP 3: PSEUDONIMIZACJA I ANONIMIZACJA ===")

        # 3a. Pseudonimizacja (Zamiana ID/Nazwiska na token UUID)
        # Zakładamy, że pierwsza kolumna to ID lub Nazwa
        id_col = df.columns[0]
        print(f"Pseudonimizacja kolumny identyfikacyjnej: '{id_col}'")

        # Tworzymy mapę prawdziwe_id -> token (w bezpiecznej bazie byłaby to osobna tabela)
        pseudo_map = {real_id: str(uuid.uuid4()) for real_id in df[id_col].unique()}
        df_secure['Patient_Token'] = df[id_col].map(pseudo_map)

        # 3b. Generalizacja (np. Wiek -> Przedział wiekowy)
        age_col = None
        for col in df.columns:
            if 'age' in col.lower() or 'wiek' in col.lower():
                age_col = col
                break

        if age_col:
            print(f"Generalizacja kolumny: '{age_col}'")
            df_secure['Age_Group'] = df[age_col].apply(lambda x: f"{(int(x)//10)*10}-{(int(x)//10)*10 + 9}")
        else:
            print("Nie znaleziono kolumny wieku do generalizacji.")

        # Wyświetlenie efektów
        cols_to_show = ['Patient_Token']
        if age_col: cols_to_show.append('Age_Group')
        cols_to_show.append(f'{target_col}_encrypted')

        print("Dane po anonimizacji (widok dla analityka):")
        display(df_secure[cols_to_show].head())

        # === ETAP 4: KONTROLA DOSTĘPU (RBAC) I AUDYT ===
        print("\n=== ETAP 4: KONTROLA DOSTĘPU (RBAC) ===")

        class MedicalDataSystem:
            def __init__(self, raw_df, secure_df):
                self.raw_df = raw_df
                self.secure_df = secure_df

            def access_data(self, role):
                log_access(role, "Ządanie dostępu do danych", "PRZETWARZANIE")

                if role == 'Administrator':
                    # Widzi wszystko
                    log_access(role, "Pobranie pełnych danych", "SUKCES")
                    return self.raw_df

                elif role == 'Lekarz':
                    # Widzi dane kliniczne i zaszyfrowane, ale potrzebuje klucza do odszyfrowania (tu symulacja)
                    # Lekarz widzi pseudonimy i oryginalne wyniki badań, ale np. bez danych administracyjnych
                    cols = [c for c in self.raw_df.columns if c != f'{target_col}_encrypted']
                    log_access(role, "Pobranie danych klinicznych", "SUKCES")
                    return self.raw_df[cols] # W uproszczeniu

                elif role == 'Analityk':
                    # Widzi TYLKO dane zanonimizowane
                    cols = ['Patient_Token']
                    if 'Age_Group' in self.secure_df.columns: cols.append('Age_Group')
                    # Dodajemy inne parametry numeryczne (np. cholesterol)
                    num_cols = self.raw_df.select_dtypes(include=['number']).columns.tolist()
                    if age_col and age_col in num_cols: num_cols.remove(age_col) # Usuwamy surowy wiek

                    final_cols = cols + num_cols
                    log_access(role, "Pobranie danych zanonimizowanych", "SUKCES")
                    return self.secure_df[final_cols]

                else:
                    log_access(role, "Nieautoryzowana próba dostępu", "ODMOWA")
                    return "Brak uprawnień"

        system = MedicalDataSystem(df, df_secure)

        print("\n--- Symulacja dostępu: Rola ANALITYK ---")
        display(system.access_data('Analityk').head())

        print("\n--- Symulacja dostępu: Rola HACKER ---")
        print(system.access_data('Hacker'))

        # === ETAP 5: PORÓWNANIE ALGORYTMÓW HASZUJĄCYCH (Zadanie 6) ===
        print("\n=== ETAP 5: PORÓWNANIE ALGORYTMÓW HASZUJĄCYCH ===")

        test_data = b"Pacjent Jan Kowalski, Diagnoza: Nadcisnienie"
        algorithms = ['md5', 'sha1', 'sha256', 'sha512', 'sha3_256']

        results = []

        for algo_name in algorithms:
            start_time = time.time()
            # Wykonujemy haszowanie 100,000 razy żeby zmierzyć czas
            for _ in range(100000):
                h = hashlib.new(algo_name)
                h.update(test_data)
                digest = h.hexdigest()
            end_time = time.time()

            results.append({
                'Algorytm': algo_name,
                'Długość bitowa': len(digest) * 4, # hex char = 4 bity
                'Czas (100k operacji) [s]': round(end_time - start_time, 4),
                'Przykład skrótu': digest[:20] + "..."
            })

        df_hash = pd.DataFrame(results)
        display(df_hash)

=== ETAP 1: WCZYTYWANIE DANYCH ===
Proszę wybrać plik CSV z lokalnego dysku:


Saving health_measurements.csv to health_measurements.csv

Sukces! Wczytano plik: health_measurements.csv
Rozmiar danych: (5, 7)

Podgląd danych surowych:


,timestamp,user_id,age,bmi,glucose,systolic_bp,diastolic_bp
0,2025-12-20 11:56:45,1,40,24.0,95,120,80
1,NaN,2,30,22.0,95,120,80
2,NaN,user_1,40,24.0,95,120,80
3,NaN,3,47,25.0,95,126,89
4,2025-12-20T12:22:15,4,20,22.0,99,110,90



=== ETAP 2: SZYFROWANIE I INTEGRALNOŚĆ ===
Wygenerowano klucz symetryczny: ofe85KGADp...
Szyfrowanie kolumny: 'user_id'


,user_id,user_id_encrypted
0,1,gAAAAABphiSMAUutvnwJXMHsO0C8CBpik1i_lTI9fKZQox...
1,2,gAAAAABphiSMInJP_FS0FbnixC0UuMVQ_j8env6pHzdQc9...
2,user_1,gAAAAABphiSMG8i6zFjfgjRMXbXiLiiTgcG6nphYZJsvEv...
3,3,gAAAAABphiSM_JbYm2xUYwm2QR8fofOMSbEGfwknUN8lDO...
4,4,gAAAAABphiSMiOXMhLndLUdA4KA-O8h4FPj98BRRLTWDhi...


...

,Patient_Token,Age_Group,user_id_encrypted
0,005fbc22-2ed9-49cd-9721-9e10dd6d8611,40-49,gAAAAABphiSMAUutvnwJXMHsO0C8CBpik1i_lTI9fKZQox...
1,3941758f-7db4-4835-9dd4-31091e1ea191,30-39,gAAAAABphiSMInJP_FS0FbnixC0UuMVQ_j8env6pHzdQc9...
2,3941758f-7db4-4835-9dd4-31091e1ea191,40-49,gAAAAABphiSMG8i6zFjfgjRMXbXiLiiTgcG6nphYZJsvEv...
3,3941758f-7db4-4835-9dd4-31091e1ea191,40-49,gAAAAABphiSM_JbYm2xUYwm2QR8fofOMSbEGfwknUN8lDO...
4,933fcb45-9fe9-4ac9-99bc-46ec20e15fc2,20-29,gAAAAABphiSMiOXMhLndLUdA4KA-O8h4FPj98BRRLTWDhi...



=== ETAP 4: KONTROLA DOSTĘPU (RBAC) ===

--- Symulacja dostępu: Rola ANALITYK ---
[AUDYT] Rola: Analityk | Akcja: Ządanie dostępu do danych | Status: PRZETWARZANIE
[AUDYT] Rola: Analityk | Akcja: Pobranie danych zanonimizowanych | Status: SUKCES


,Patient_Token,Age_Group,bmi,glucose,systolic_bp,diastolic_bp
0,005fbc22-2ed9-49cd-9721-9e10dd6d8611,40-49,24.0,95,120,80
1,3941758f-7db4-4835-9dd4-31091e1ea191,30-39,22.0,95,120,80
2,3941758f-7db4-4835-9dd4-31091e1ea191,40-49,24.0,95,120,80
3,3941758f-7db4-4835-9dd4-31091e1ea191,40-49,25.0,95,126,89
4,933fcb45-9fe9-4ac9-99bc-46ec20e15fc2,20-29,22.0,99,110,90



--- Symulacja dostępu: Rola HACKER ---
[AUDYT] Rola: Hacker | Akcja: Ządanie dostępu do danych | Status: PRZETWARZANIE
[AUDYT] Rola: Hacker | Akcja: Nieautoryzowana próba dostępu | Status: ODMOWA
Brak uprawnień

=== ETAP 5: PORÓWNANIE ALGORYTMÓW HASZUJĄCYCH ===


,Algorytm,Długość bitowa,Czas (100k operacji) [s],Przykład skrótu
0,md5,128,0.1244,acc7ac4efaa9e04bb760...
1,sha1,160,0.1356,731ab827dce1b66966b0...
2,sha256,256,0.1953,fa07a7822f79feb77dac...
3,sha512,512,0.2845,03c4c2e57f5f3dbac3b7...
4,sha3_256,256,0.3160,61a98950d48aad3216ad...
